In [1]:
import sys

sys.path.append('../..')

In [2]:
import dill

loaded_data = None

with open('../simulations/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

keys = loaded_data.keys()
keys.sort()
print keys

['N', 'credit', 'randomization', 'underlyings']


In [3]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 15*step

maturity = loaded_data["randomization"]["maturity"]

#Discount

In [4]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [5]:
udlyings = loaded_data["underlyings"]

bm = udlyings[0]
time_grid = bm.time

# Derivative

In [6]:
derivatives_nb = 1

In [7]:
from finance.products.european.assetswap import (
    SwapContract,
)

swap_delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(bm, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 =  0.0


# Exposure

In [8]:
from risk.exposures import EuropeanQuantileBrownianExposure

quantile_im = 0.70
quantile_df = 0.80

im_drift = 0.3
im_vol = bm.vols[0][0]

exposure = EuropeanQuantileBrownianExposure(swap, discount, im_drift, im_vol)

# Indexes stuffs

## All idiosyncratic ids

In [9]:
copula = loaded_data["credit"]["copula"]

all_ids = [list(sub[0])[0] for sub in copula.subsets if len(sub[0]) == 1]
obligors_nb = len(all_ids)

print "Obligors number: %s\n"%obligors_nb
print "All ids (including those that have a zero weight): %s"%all_ids

Obligors number: 125

All ids (including those that have a zero weight): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]


## Ids of $B$ and $C$

In [10]:
b_id = 113

bc_ids = loaded_data["credit"]["bc_ids"]
c_ids = bc_ids[:]
c_ids.remove(b_id)

c_ids.sort()

print "Bank id: %s\n"%b_id
print "Counterparties ids: %s"%c_ids

Bank id: 113

Counterparties ids: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124]


## Subsets indexes of the copula for $B$ and $C$

In [11]:
armaggeddon_subset_index = 129

In [12]:
bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

b_subsets_indexes = copula.get_indexes_including(b_id)
b_subsets_indexes.remove(armaggeddon_subset_index)

c_subsets_indexes = bc_subsets_indexes[:]
for idx in b_subsets_indexes:
    c_subsets_indexes.remove(idx)
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n"%bc_subsets_indexes
print "Bank subsets indexes: %s\n"%b_subsets_indexes
print "Counterparties subsets indexes: %s"%c_subsets_indexes

Subsets that generated default times: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124, 125, 126, 127, 128]

Bank subsets indexes: [113, 128]

Counterparties subsets indexes: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124, 125, 126, 127]


# Portfolio construction

In [13]:
import numpy as np

p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [14]:
from finance.portfolio import CCPPortfolio

pos = CCPPortfolio.generate_1_vs_all_positions(b_id, bc_ids, obligors_nb)
positions = np.array(pos*notional).reshape(pos.size, 1)

In [15]:
port = CCPPortfolio(positions, [swap], [exposure])

# VM, IM and DF accounts

In [16]:
from ccp.accounts import Accounts, DFAccounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)
df_accounts = DFAccounts(states, derivatives_nb)

# Skin in the game

In [17]:
from ccp.sig import SkinInTheGame

sig = SkinInTheGame(0.)

# Regulatory capital

In [18]:
from risk.basel import CCPRegulatoryCapital

# We first set a beta factor to 30% (recommandation of Dosseur)
beta = 0.3

regul_cap = CCPRegulatoryCapital(vm_accounts, im_accounts, df_accounts, sig, beta, port)

# Funding and capital parameters

In [19]:
lambda_ = 0.
k_ = 0.1
c_0 = -.0020

bank_recov = 0.4

# Creation of the different taks

In [20]:
class ITask(object):
    level = None
    
    def perform(self, **kwargs):
        raise NotImplementedError()       

In [21]:
class PutMembersAtDefaultTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__states = kwargs["states"]
        self.potential_defaulters = kwargs["defaulters"]
        self.defaulters = []
        
    def perform(self, **kwargs):
        self.__tau = kwargs["t"]
        
        for i in self.potential_defaulters:
            for j in list(i):
                if self.__states.is_alive(j):
                    self.defaulters.append(j)
                    self.__states.die(j)
            
        self.defaulters.sort()

In [22]:
class LiquidatePortfolioTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__cm_index = kwargs["bank_index"]
        
        self.__default_task = kwargs["default_task"]
                
        self.__port = kwargs['portfolio']
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
                
        self.__discount = kwargs["discount"]
        self.__sig = kwargs['sig']
        
        self.results = {}
        
    def perform(self, **kwargs):
        self.__tau_delta = kwargs["t"]
        
        self.new_defaulters_ids = self.__default_task.defaulters
        
        contract_prices = [d.price(self.__tau_delta) for d in self.__port.derivatives]
        p_tau_delta = port.compute_value(contract_prices)[self.new_defaulters_ids, :]
        p_tau_delta = np.sum(p_tau_delta, axis=1)
        
        q_tau_delta = p_tau_delta
        
        gamma_tau_delta = self.__vm_accounts.amounts + self.__im_accounts.amounts + self.__df_accounts.amounts
        gamma_tau_delta = gamma_tau_delta[self.new_defaulters_ids, :]
        gamma_tau_delta = np.sum(gamma_tau_delta, axis=1)
        
        loss_tau_delta = q_tau_delta - gamma_tau_delta
        
        remain_collat_loss = 0.
        for l in loss_tau_delta:
            remain_collat_loss += np.maximum(l, 0)
            
        remaining_loss = self.__sig.handle_breach(remain_collat_loss)
        
        frac = self.__df_accounts.get_amount(self.__cm_index) / self.__df_accounts.total_default_fund()
        
        partial_loss = frac*remaining_loss
        
        tmp_cva = self.__discount(self.__tau_delta) * partial_loss
        self.results["cva"] = tmp_cva

In [23]:
class FillVMandIMTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self._delta = kwargs["delta"]
        
        self._port = kwargs['portfolio']        
        self._vm_accounts = kwargs['vm_accounts']
        self._im_accounts = kwargs['im_accounts']
        
        self._quantile_im = kwargs['quantile_im']
        
        self._zeros = np.zeros(self._port.derivatives.size)
        self._zeros.fill(1e-5)
        
    def perform(self, **kwargs):
        self._tau_minus = kwargs["t"]
        
        #############
        ## VM part ##
        #############        
        contract_prices = [d.price(self._tau_minus) for d in self._port.derivatives]
        p_tau_minus = self._port.compute_value(contract_prices)
        
        for ii, p_and_l in enumerate(p_tau_minus):
            self._vm_accounts.put_amounts(ii, p_and_l)
        
        #############
        ## IM part ##
        #############        
        exposures = self._port.compute_exposure(self._tau_minus, risk_period=self._delta, conf_level=self._quantile_im)
        for ii, ee in enumerate(exposures):
            self._im_accounts.put_amounts(ii, np.maximum(ee, self._zeros).tolist())

In [24]:
class FillIMVMandDFTask(FillVMandIMTask):
    def __init__(self, **kwargs):
        super(FillIMVMandDFTask, self).__init__(**kwargs)
        self._df_accounts = kwargs['df_accounts']        
        self._quantile_df = kwargs['quantile_df']
        
    def perform(self, **kwargs):
        super(FillIMVMandDFTask, self).perform(**kwargs)
        self._t = kwargs["t"]
        
        exposures = self._port.compute_exposure(self._t, risk_period=self._delta, conf_level=self._quantile_df)
        for ii, (ee, im) in enumerate(zip(exposures, self._im_accounts.amounts)):
            self._df_accounts.put_amounts(ii, np.maximum(ee - im, self._zeros).tolist())

In [25]:
class ReFillSigTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self.__sig = kwargs['sig']
    
    def perform(self, **kwargs):
        self.__sig.recover()

In [26]:
class ComputeDFKVATask(ITask):
    level = 1
    
    def __init__(self, **kwargs):        
        self.__cm_index = kwargs["bank_index"]
        
        self.__delta = kwargs["delta"]
        self.__exp_distrib = kwargs["exp_distrib"]
        
        self.__copula = kwargs["copula"]
        self.__cm_subset_indexes = kwargs["bank_subsets_indexes"]
        
        self.__port = kwargs['portfolio']        
        self.__time_grid = self.__port.derivatives[0].underlying.time
        
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
        
        self.__discount = kwargs["discount"]
        
        self.__regul_cap = kwargs['regul_capital']
        
        self.__c_zeta = kwargs["c_zeta"]
        self.__cm_recov = kwargs["bank_recov"]
        self.__lambda_zeta = kwargs["lambda_zeta"]
        self.__k_zeta = kwargs["k_zeta"]
        
        self.__collat_task = FillIMVMandDFTask(delta=delta, portfolio=self.__port,
                                               vm_accounts=self.__vm_accounts,
                                               im_accounts=self.__im_accounts, 
                                               quantile_im=kwargs['quantile_im'],
                                               df_accounts=self.__df_accounts, 
                                               quantile_df=kwargs['quantile_df'])
        
        self.results = {}
    
    def perform(self, **kwargs):
        self.__zeta = kwargs["t"]
        self.__zeta_delta = time_offseter(self.__zeta+self.__delta, self.__time_grid, True) 
        
        inv_pdf = 1./self.__exp_distrib.pdf(zeta)
        
        beta_zeta = self.__discount(self.__zeta)
        
        self.__collat_task.perform(t=self.__zeta)
        
        ################
        ### DVA part ###
        ################        
        contract_prices = [d.price(self.__zeta_delta) for d in self.__port.derivatives]
        p_zeta_delta = port.compute_value(contract_prices)
        q_zeta_delta = p_zeta_delta
        collateral = self.__vm_accounts.amounts + self.__im_accounts.amounts + self.__df_accounts.amounts        
        
        q_zeta_delta = q_zeta_delta[self.__cm_index].sum()
        collateral = collateral[self.__cm_index].sum()
        
        loss = np.maximum(q_zeta_delta - collateral, 0.)
        
        gamma_zeta = self.__copula.tot_gamma(self.__zeta, subsets_indexes=self.__cm_subset_indexes)
        
        self.results["dva"] = inv_pdf*self.__discount(self.__zeta_delta)*gamma_zeta*loss
        
        ################
        ### FVA part ###
        ################
        contract_prices = [d.price(self.__zeta) for d in self.__port.derivatives]
        p_zeta = port.compute_value(contract_prices)
        fund = p_zeta[self.__cm_index].sum() - collateral
        
        bar_lambda_zeta = 0.5*(1-self.__cm_recov)*gamma_zeta 
        
        tmp_fva = -self.__c_zeta * collateral
        tmp_fva += -bar_lambda_zeta * np.minimum(fund, 0.)
        tmp_fva += -self.__lambda_zeta * np.maximum(fund, 0.)
        
        self.results["fva"] = inv_pdf*beta_zeta*tmp_fva
        
        ################
        ### KVA part ###
        ################        
        self.results["kva"] = inv_pdf*self.__k_zeta*beta_zeta*self.__regul_cap.compute_kcm(self.__cm_index, self.__zeta)

In [27]:
def merge_dict(actual_dict, to_add):
    for k, v in to_add.iteritems():
        if k not in actual_dict:
            actual_dict[k] = []
            
        actual_dict[k].append(v)
        
def sort_tasks_dict(a_dict):
    for k, v in a_dict.iteritems():
        v.sort(key=lambda x: x.level)

#MC loop

In [28]:
from utils import time_offseter

In [29]:
N = loaded_data["N"]

In [30]:
results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva": {"sum": 0., "sum2": 0.}}

In [31]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [32]:
sig_time_grid = [t for t in time_grid if t.is_integer()]
df_time_grid = [t for t in time_grid if (12*t).is_integer()]

In [33]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]
    b_min_df_time = default_times[b_df_times_indexes].min()
    bar_tau = min(b_min_df_time, maturity)
    
    df_times_dict = {
        t: PutMembersAtDefaultTask(states=states, defaulters=copula.subsets[bc_subsets_indexes[j]])
        for j, t in enumerate(default_times) if t < bar_tau
    }
    
    liq_times_dict = {
        time_offseter(t+delta, time_grid): 
        LiquidatePortfolioTask(bank_index=b_id, default_task=task, 
                               portfolio=port, discount=discount, sig=sig,
                               vm_accounts=vm_accounts, im_accounts=im_accounts, 
                               df_accounts=df_accounts)
        for t, task in df_times_dict.iteritems() if t+delta < bar_tau        
    }
    
    tau_minus_times_dict = {
        time_offseter(t-delta, time_grid, True):
        FillVMandIMTask(delta=delta, portfolio=port, 
                        vm_accounts=vm_accounts, im_accounts=im_accounts,
                        quantile_im=quantile_im)        
        for t in df_times_dict.keys()
    }
    
    dfund_times = [time_offseter(t, df_time_grid, True) for t in df_times_dict.keys()]    
    dfund_times_dict = {
        t: FillIMVMandDFTask(delta=delta, portfolio=port, 
                             vm_accounts=vm_accounts, 
                             im_accounts=im_accounts, quantile_im=quantile_im,
                             df_accounts=df_accounts, quantile_df=quantile_df)
        for t in dfund_times
    }

    sig_times = {
        t: ReFillSigTask(sig=sig)
        for t in sig_time_grid if t < zeta
    }

    dfkva_time = {
        t: ComputeDFKVATask(bank_index=b_id, exp_distrib=exp_distrib, 
                            copula=copula, portfolio=port, delta=delta,
                            bank_subsets_indexes=b_subsets_indexes,
                            vm_accounts=vm_accounts, 
                            im_accounts=im_accounts, quantile_im=quantile_im,
                            df_accounts=df_accounts, quantile_df=quantile_df,
                            discount=discount, 
                            regul_capital=regul_cap, c_zeta=c_0, 
                            lambda_zeta=lambda_, k_zeta=k_, bank_recov=bank_recov)
        for t in [zeta] if zeta <= bar_tau
    }
    
    current_times_dict = dict()
    all_dicts = [df_times_dict, liq_times_dict, tau_minus_times_dict, \
                 dfund_times_dict, sig_times, dfkva_time]
    
    for dd in all_dicts:
        merge_dict(current_times_dict, dd)
    
    sort_tasks_dict(current_times_dict)
    
    current_times = current_times_dict.keys()
    current_times.sort()
    
    for t in current_times:
        tasks = current_times_dict[t]
        for task in tasks:
            task.perform(t=t)
        
    cva_, dva_, fva_, kva_ = [0. for _ in range(4)]
    for k, v in liq_times_dict.iteritems():
        cva_ += v.results["cva"]
        
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    for k, v in dfkva_time.iteritems():
        dva_ += v.results["dva"]
        fva_ += v.results["fva"]
        kva_ += v.results["kva"]
        
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva"]["sum"] += kva_
    results_globs["kva"]["sum2"] += kva_**2
    
toc = time.time()

#Results

In [34]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], \
                                                         copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "\nConfidence level used for VM+IM: %.2f"%quantile_im
print "Confidence level used for VM+IM+DF: %.2f"%quantile_df

Results for 20000 iterations (475.648000002 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 113 that belongs to the following MO copula subsets:
- [113] with pillars [3 5] and intensity [ 0.00374407  0.00339649]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

Counterparties indexes:
[0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124]

-----------------------------------------------------

Counterparty index 0 belongs to the following MO copula subses:
- [0] with pillars [3 

In [35]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)    

Results of the global xVA:

The CVA for the bank lies in [0.003704, 0.004277]
The DVA for the bank lies in [0.000399, 0.000442]
The FVA for the bank lies in [0.013442, 0.013749]
The KVA for the bank lies in [0.068744, 0.070897]
